# Expert Trajectory Generation

In [5]:
#imports for simulation
from modules.runtime.scenario.scenario_generation.interaction_dataset_scenario_generation import \
    InteractionDatasetScenarioGeneration
from modules.runtime.commons.parameters import ParameterServer
from modules.runtime.viewer.matplotlib_viewer import MPViewer
from modules.runtime.viewer.video_renderer import VideoRenderer
import os
import os.path
import argparse
import matplotlib.pyplot as plt

from IPython.display import clear_output

#debugging
import sys

#imports for observers
from bark_ml.python.bark_ml_library.observers import NearestObserver

#imports for -pkl file
import pickle


In [6]:
#Set path to interaction dataset
#my current path is /home/marvin/Repositories/interaction_dataset
path_to_interaction_dataset = os.path.join(os.path.expanduser('~'), "praktikum/interaction_dataset")
if not os.path.exists(path_to_interaction_dataset):
    raise ValueError('Interaction dataset not found at location:', path_to_interaction_dataset)

In [7]:
# Lets define the secnario:
param_server = ParameterServer()
param_server["Scenario"]["Generation"]["InteractionDatasetScenarioGeneration"]["MapFilename"] = os.path.join(path_to_interaction_dataset, "DR_DEU_Merging_MT/map/DR_DEU_Merging_MT_v01_shifted.xodr")
param_server["Scenario"]["Generation"]["InteractionDatasetScenarioGeneration"]["TrackFilename"] = os.path.join(path_to_interaction_dataset, "DR_DEU_Merging_MT/tracks/vehicle_tracks_013.csv")
param_server["Scenario"]["Generation"]["InteractionDatasetScenarioGeneration"]["TrackIds"] = [63,64,65,66,67,68]
param_server["Scenario"]["Generation"]["InteractionDatasetScenarioGeneration"]["StartTs"] = 232000
param_server["Scenario"]["Generation"]["InteractionDatasetScenarioGeneration"]["EndTs"] = 259000
param_server["Scenario"]["Generation"]["InteractionDatasetScenarioGeneration"]["EgoTrackId"] = 66

In [8]:
# Create the scenario
scenario_generation = InteractionDatasetScenarioGeneration(num_scenarios=1, random_seed=0, params=param_server)
scenario = scenario_generation.create_single_scenario()
#following warning will occur:
"""
FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if lane.find("userData"):
"""

'\nFutureWarning: The behavior of this method will change in future versions. Use specific \'len(elem)\' or \'elem is not None\' test instead.\n  if lane.find("userData"):\n'

In [9]:
# Initialize
sim_step_time = 0.2

In [10]:
# Run the simulation for couple of steps
world_state = scenario.get_world_state()

In [11]:
world_state.DoPlanning(sim_step_time)
world_state.DoExecution(sim_step_time)

In [12]:
list_observed_worlds = world_state.Observe(list(range(1,87)))
print(list_observed_worlds[1].ego_agent.id)
print(list_observed_worlds[1].time)
print(dir(list_observed_worlds[1].ego_agent.behavior_model))

64
0.20000000298023224
['ActionToBehavior', 'Clone', 'GetLastAction', 'Plan', 'SetLastAction', 'SetLastTrajectory', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', 'last_trajectory', 'static_trajectory']


In [13]:
#gives as all accessible methods for the world_state
print(type(world_state))
dir(world_state)

<class 'bark.world.World'>


['AddAgent',
 'AddEvaluator',
 'AddObject',
 'ClearEvaluators',
 'Copy',
 'DoExecution',
 'DoPlanning',
 'Evaluate',
 'FillWorldFromCarla',
 'GetAgent',
 'GetNearestAgents',
 'GetParams',
 'Observe',
 'PlanAgents',
 'SetMap',
 'Step',
 'UpdateAgentRTree',
 'WorldExecutionAtTime',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'agents',
 'agents_valid',
 'bounding_box',
 'evaluators',
 'map',
 'objects',
 'time']

In [14]:
#get a list of the observed worlds for the agents at the current timestep
list_observed_worlds = world_state.Observe([63,64,65,66,67,68])
#type(list_observed_worlds) : list

In [15]:
#get all methods for an observed world
observed_world=list_observed_worlds[0]
dir(observed_world)

['AddAgent',
 'AddEvaluator',
 'AddObject',
 'ClearEvaluators',
 'Copy',
 'DoExecution',
 'DoPlanning',
 'Evaluate',
 'FillWorldFromCarla',
 'GetAgent',
 'GetAgentBehind',
 'GetAgentInFront',
 'GetNearestAgents',
 'GetParams',
 'Observe',
 'PlanAgents',
 'PredictWithOthersIDM',
 'SetMap',
 'Step',
 'UpdateAgentRTree',
 'WorldExecutionAtTime',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'agents',
 'agents_valid',
 'bounding_box',
 'ego_agent',
 'ego_position',
 'ego_state',
 'evaluators',
 'lane_corridor',
 'map',
 'objects',
 'other_agents',
 'road_corridor',
 'time']

In [16]:
#Unfortunately the method GetEgoAgentID is not accessible for some reasons
#to get the agent id we do a small workaround and go over the ego agent and then take his id
observed_world.ego_agent.id

63

## What does the .pkl file contain?

dict with the keys:
 - "obs" 
 - "next_obs" 
 - "action" 
 - "rew" 
 - "done"
 
every key holds a numpy.ndarray of Shape (rows ,  cols), where the number of
- rows: are the same for all keys
- cols: obs = next_obs = flex , action = flex , rew = done = 1

done is 1 if epoche is finished and 0 otherwise

## What does the Scenario contain (Track_013)

Timestamp in ms: starting at 100ms  
Every 100ms is next observation -> 100ms frequence  
StartTs:100ms  
EndTs: 327300ms = 327,3s  
cars: 1, ... , 86

In [21]:
#Create the complete scenario for a track filename i.e. vehicle_tracks_013.csv

#file-specific information

#id=18 is not available
car_id_list=list(range(1,18))+list(range(19,87))
StartTs=100
EndTs=327300

#Scenario - Set up
param_server = ParameterServer()
param_server["Scenario"]["Generation"]["InteractionDatasetScenarioGeneration"]["MapFilename"] = os.path.join(path_to_interaction_dataset, "DR_DEU_Merging_MT/map/DR_DEU_Merging_MT_v01_shifted.xodr")
param_server["Scenario"]["Generation"]["InteractionDatasetScenarioGeneration"]["TrackFilename"] = os.path.join(path_to_interaction_dataset, "DR_DEU_Merging_MT/tracks/vehicle_tracks_013.csv")
param_server["Scenario"]["Generation"]["InteractionDatasetScenarioGeneration"]["TrackIds"] = car_id_list
param_server["Scenario"]["Generation"]["InteractionDatasetScenarioGeneration"]["StartTs"] = StartTs
param_server["Scenario"]["Generation"]["InteractionDatasetScenarioGeneration"]["EndTs"] = EndTs
param_server["Scenario"]["Generation"]["InteractionDatasetScenarioGeneration"]["EgoTrackId"] = 66

#Initialize observer
observer = NearestObserver(param_server)

In [22]:
scenario_generation = InteractionDatasetScenarioGeneration(num_scenarios=1, random_seed=0, params=param_server)
scenario = scenario_generation.create_single_scenario()

In [23]:
#Simulation-specific configurations
speed_factor = 1
sim_step_time = 100*speed_factor/1000
sim_steps = int((EndTs-StartTs)/(sim_step_time*1000))

In [24]:
#Initialize the dict

#we cant use np.darray directly as it doesnt support dynamic length of a vector
#but it can be easily transformend afterwards with array = np.array(list)
#list has to be a sequence of lists -> so one list per timestamp appended to the whole list to create two-dim vector

expert_traj = {}
for agent_id in car_id_list:
    expert_traj[agent_id] = {'obs' : [],
                             'next_obs': [],
                             'action': [],
                             'done': [],
                             'time': [],
                             'merge': []
                            }

In [29]:
print(expert_traj)

{1: {'obs': [], 'next_obs': [], 'action': [], 'done': [], 'time': [], 'merge': []}, 2: {'obs': [], 'next_obs': [], 'action': [], 'done': [], 'time': [], 'merge': []}, 3: {'obs': [], 'next_obs': [], 'action': [], 'done': [], 'time': [], 'merge': []}, 4: {'obs': [], 'next_obs': [], 'action': [], 'done': [], 'time': [], 'merge': []}, 5: {'obs': [], 'next_obs': [], 'action': [], 'done': [], 'time': [], 'merge': []}, 6: {'obs': [], 'next_obs': [], 'action': [], 'done': [], 'time': [], 'merge': []}, 7: {'obs': [], 'next_obs': [], 'action': [], 'done': [], 'time': [], 'merge': []}, 8: {'obs': [], 'next_obs': [], 'action': [], 'done': [], 'time': [], 'merge': []}, 9: {'obs': [], 'next_obs': [], 'action': [], 'done': [], 'time': [], 'merge': []}, 10: {'obs': [], 'next_obs': [], 'action': [], 'done': [], 'time': [], 'merge': []}, 11: {'obs': [], 'next_obs': [], 'action': [], 'done': [], 'time': [], 'merge': []}, 12: {'obs': [], 'next_obs': [], 'action': [], 'done': [], 'time': [], 'merge': []}, 

In [30]:
def Calc_Action(obs,next_obs,delta_t):
    """
    @Pedro:
    here comes your function to get the obs state by observer

    Input: 
        obs : list of observations as a sequence of lists [[x,y,theta,v],[x,y,theta,v]]
        next_obs : list of observations as a sequence of lists [[x,y,theta,v],[x,y,theta,v]]
        delta_t : the time between two consecutive frames
    Output: list with actions [steering_rate,accelaration_rate]
    """
    action = [0,0]
    
    # calculate steering angle (angular velocity)
    action[0] = (next_obs[2] - obs[2]) / delta_t
    
    #just for debugging
    action[1] = (next_obs[3] - obs[3]) / delta_t
    
    return action

In [27]:
def Create_Obs_State(obs_world):
    """
    @Pedro:
    here comes your function to get the obs state by observer

    Input: obs_world
    Output: list with all information [xpos,ypos,vel,theta,.....]
    """
    
    
    #just for debugging
    obs_state = [0]
    
    return obs_state

In [31]:
def Append_timestamp_trajectory (exp_traj,obs_world_list,world):
    """Append the current timestamp trajectory of active agents to expert trajectory
    """
    agents_valid = list(world.agents_valid.keys())
    
    for obs_world in obs_world_list:
    
        agent_id = obs_world.ego_agent.id
        
        if agent_id in agents_valid:
            if not exp_traj[agent_id]['obs']:
                exp_traj[agent_id]['obs'].append(Create_Obs_State(obs_world))
                #

                exp_traj[agent_id]['time'].append([world.time])

                #TBD: check Road-Corridor for first time - for some reasons there is not always a lane available
                try:
                    exp_traj[agent_id]['merge'].append(obs_world.lane_corridor.center_line.bounding_box[0].x()>900)
                except:
                    pass
                                       

            else:

                #@Pedro
                #TBD: add the oberserver + possible return transformation
                #TBD: add a function to Calculate the action based on two observation
                exp_traj[agent_id]['obs'].append(Create_Obs_State(obs_world))    
                exp_traj[agent_id]['next_obs'].append(exp_traj[agent_id]['obs'][-1])
                exp_traj[agent_id]['action'].append(Calc_Action(exp_traj[agent_id]['obs'][-1],\
                                                                exp_traj[agent_id]['next_obs'][-1], \
                                                                exp_traj[agent_id]['time'][-1][0] - exp_traj[agent_id]['time'][-2][0]))
                #

                #Check the road corridor if not already defined
                if not exp_traj[agent_id]['merge']:
                    try:
                        exp_traj[agent_id]['merge'].append(obs_world.lane_corridor.center_line.bounding_box[0].x()>900)
                    except:
                        pass
                        
                        
                exp_traj[agent_id]['done'].append([0])         
                exp_traj[agent_id]['time'].append([world.time])
         
    return exp_traj

In [32]:
#Simulation

world_state = scenario.get_world_state()

for _ in range(0, sim_steps):
    world_state.DoPlanning(sim_step_time)
    world_state.DoExecution(sim_step_time)
    observed_world_list = world_state.Observe(car_id_list)
    expert_traj=Append_timestamp_trajectory(expert_traj,observed_world_list,world_state)

for agent_id in expert_traj:
    
    #@Pedro
    #add zeros depending on the obs-state size
    expert_traj[agent_id]['next_obs'].append([0])
    #add zeros depending on the action-state size
    expert_traj[agent_id]['action'].append([0,0])
    #or we duplicate the 'obs' and 'action' so no difficult sizing has to be done....
    
    expert_traj[agent_id]['done'].append([1])        


IndexError: list index out of range

In [ ]:
#Store to dict
#@Pedro can you implement this?

## What do you think pedro?

In [ ]:
"""
for _ in range(0, sim_time_steps):
    world_state.DoPlanning(sim_step_time)
    fig = plt.figure(figsize=[10, 10])
    viewer = MPViewer(params=param_server, use_world_bounds=True, axis=fig.gca())
    viewer.drawWorld(world_state, scenario._eval_agent_ids)
    world_state.DoExecution(sim_step_time)
    clear_output(wait=True)
"""